##  1. Build a Flask app that scrapes data from multiple websites and displays it on your site.
## You can try to scrap websites like youtube , amazon and show data on output pages and deploy it on cloud platform 

In [ ]:
Step 1: Set Up the Flask Application

app.py (Main Flask Application):

from flask import Flask, render_template, request
import requests
from bs4 import BeautifulSoup

app = Flask(__name__)

# Scrape data from YouTube (Example: Trending videos)
def scrape_youtube():
    url = 'https://www.youtube.com/feed/trending'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    videos = []
    for video in soup.find_all('a', {'class': 'yt-simple-endpoint style-scope ytd-video-renderer'}):
        title = video.text
        link = 'https://www.youtube.com' + video['href']
        videos.append({'title': title, 'link': link})
    return videos

# Scrape data from Amazon (Example: Best-selling books)
def scrape_amazon():
    headers = {"User-Agent": "Mozilla/5.0"}
    url = 'https://www.amazon.com/best-sellers-books-Amazon/zgbs/books'
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    books = []
    for book in soup.find_all('div', {'class': 'p13n-sc-truncate-desktop-type2 p13n-sc-truncated'}):
        books.append(book.text.strip())
    return books

# Flask Routes
@app.route('/')
def home():
    return render_template('index.html')

@app.route('/scrape', methods=['POST'])
def scrape():
    if request.form['website'] == 'youtube':
        data = scrape_youtube()
        return render_template('result.html', data=data, site="YouTube")
    elif request.form['website'] == 'amazon':
        data = scrape_amazon()
        return render_template('result.html', data=data, site="Amazon")
    return render_template('index.html')

if __name__ == '__main__':
    app.run(debug=True)


Step 2: Create HTML Templates
index.html:

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Web Scraper</title>
</head>
<body>
    <h1>Web Scraper</h1>
    <form action="/scrape" method="POST">
        <label for="website">Select Website:</label>
        <select name="website" id="website">
            <option value="youtube">YouTube</option>
            <option value="amazon">Amazon</option>
        </select>
        <input type="submit" value="Scrape Data">
    </form>
</body>
</html>

result.html:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Scraped Data from {{ site }}</title>
</head>
<body>
    <h1>Scraped Data from {{ site }}</h1>
    <ul>
        {% for item in data %}
            <li>{{ item }}</li>
        {% endfor %}
    </ul>
    <a href="/">Go Back</a>
</body>
</html>
Step 3: Deployment to Heroku
Create requirements.txt:

Ensure you include the necessary dependencies in requirements.txt:

Flask==2.1.0
requests==2.26.0
beautifulsoup4==4.9.3
Create Procfile:

This is required by Heroku to run the app.

makefile

web: gunicorn app:app
Deploy to Heroku:

Install the Heroku CLI: Heroku CLI Install.

Create a new Heroku project:


heroku create
Push the code to Heroku:


git init
git add .
git commit -m "Initial commit"
git push heroku master
Open the app:

heroku open
Step 4: Cloud Deployment Alternatives
If you prefer not to use Heroku, you can also deploy the app on platforms like:

AWS Elastic Beanstalk:

Package your Flask app as a zip file and deploy it to AWS Elastic Beanstalk.
Google Cloud Run:

Build a Docker container for your Flask app and deploy it using Cloud Run.
DigitalOcean:

You can host Flask apps on a basic DigitalOcean Droplet using a custom domain.
Best Practices
Handle Rate-Limiting and Legal Issues: Scraping sites like Amazon and YouTube might be against their terms of service. Always respect the robots.txt and API offerings (e.g., YouTube Data API).

Use APIs: Instead of scraping, use APIs where available. For example, YouTube has a Data API and Amazon offers APIs via AWS.

Caching: Implement caching mechanisms (e.g., using Redis) to avoid scraping too frequently and to speed up response times.

Next Steps
Add More Websites: Integrate more scraping options.
Use a Headless Browser: For more complex websites like Amazon, consider using Selenium to interact with JavaScript-heavy content.
Enhance Output: Display more detailed information (e.g., video thumbnails, prices, etc.).
User Input: Allow users to input specific queries (e.g., search YouTube for a term).
This approach provides a solid foundation for web scraping with Flask and allows easy deployment to the cloud.


##  2. Create a Flask app that consumes data from external APIs and displays it to users.
## Try to find an public API which will give you a data and based on that call it and deploy it on cloud platform

In [ ]:
Step 1: Set Up Flask Application

app.py (Main Flask Application):

from flask import Flask, render_template, request
import requests

app = Flask(__name__)

# Your OpenWeatherMap API Key
API_KEY = 'your_openweathermap_api_key'

# Function to fetch weather data from OpenWeatherMap API
def get_weather(city):
    url = f'http://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}&units=metric'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return {
            'city': data['name'],
            'temperature': data['main']['temp'],
            'description': data['weather'][0]['description'],
            'icon': data['weather'][0]['icon']
        }
    else:
        return None

# Flask Routes
@app.route('/')
def home():
    return render_template('index.html')

@app.route('/weather', methods=['POST'])
def weather():
    city = request.form['city']
    weather_data = get_weather(city)
    if weather_data:
        return render_template('weather.html', weather=weather_data)
    else:
        return render_template('error.html', city=city)

if __name__ == '__main__':
    app.run(debug=True)
Step 2: Create HTML Templates
index.html:

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Weather App</title>
</head>
<body>
    <h1>Weather App</h1>
    <form action="/weather" method="POST">
        <label for="city">Enter City:</label>
        <input type="text" id="city" name="city" required>
        <input type="submit" value="Get Weather">
    </form>
</body>
</html>
weather.html:

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Weather in {{ weather.city }}</title>
</head>
<body>
    <h1>Weather in {{ weather.city }}</h1>
    <p>Temperature: {{ weather.temperature }}°C</p>
    <p>Condition: {{ weather.description }}</p>
    <img src="http://openweathermap.org/img/w/{{ weather.icon }}.png" alt="Weather Icon">
    <a href="/">Search another city</a>
</body>
</html>
error.html:

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Error</title>
</head>
<body>
    <h1>Weather information for "{{ city }}" could not be retrieved.</h1>
    <a href="/">Try again</a>
</body>
</html>
Step 3: Deployment to Heroku
Create requirements.txt:

To make sure all dependencies are installed in the deployment environment:


pip freeze > requirements.txt
Example requirements.txt:

Flask==2.1.0
requests==2.26.0
gunicorn==20.1.0
Create Procfile:

This tells Heroku how to run the app:

makefile

web: gunicorn app:app
Deploy to Heroku:

Create a Git repository:

git init
git add .
git commit -m "Initial commit"
Create a Heroku app:

heroku create
Deploy your code to Heroku:

git push heroku master
Set the OpenWeatherMap API key as an environment variable on Heroku:

heroku config:set API_KEY=your_openweathermap_api_key
Open the app:

heroku open
Step 4: Deployment Alternatives
AWS Elastic Beanstalk:

Package your Flask app and deploy to AWS Elastic Beanstalk.
Google Cloud Run:

Build a Docker image of your Flask app and deploy it on Cloud Run.
DigitalOcean:

Host the app on a basic DigitalOcean droplet with a custom domain.
Enhancements
More APIs: Add multiple API integrations (e.g., news APIs, cryptocurrency data).
Error Handling: Improve error handling for different API failure scenarios.
CSS Styling: Enhance the UI with CSS to improve user experience.
Caching: Use caching mechanisms (e.g., Redis) to avoid making redundant API calls.
API Rate Limits: Be mindful of API rate limits, and implement mechanisms to handle exceeding limits.

##  3. Implement OAuth2 authentication to allow users to log in using their Google or Facebook accounts

In [ ]:
Step 1: Install Required Libraries
Start by installing the necessary libraries:

pip install Flask Flask-Dance Flask-Login Authlib


Step 2: Configure Google OAuth2
Create a Google OAuth Application:

Go to the Google Developers Console.
Create a new project.
Go to "Credentials" and create OAuth2 credentials.
Configure the redirect URI (e.g., http://localhost:5000/login/google/authorized for local development).
Get your Client ID and Client Secret.
Enable Google APIs:

Enable the "Google+ API" (or "People API") in the Google API Console for user information access.
Step 3: Set Up the Flask App
Here’s how to create the Flask app and integrate OAuth2 authentication using Google.

app.py

from flask import Flask, redirect, url_for, render_template
from flask_dance.contrib.google import make_google_blueprint, google
from flask_login import LoginManager, login_user, logout_user, current_user, login_required
from authlib.integrations.flask_client import OAuth
from flask_sqlalchemy import SQLAlchemy
import os

app = Flask(__name__)

# Set the secret key and OAuth credentials (Use environment variables in production)
app.secret_key = 'random_secret_key'
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///users.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

db = SQLAlchemy(app)
login_manager = LoginManager(app)
login_manager.login_view = 'google.login'

# Set up Google OAuth blueprint
blueprint = make_google_blueprint(
    client_id=os.getenv('GOOGLE_CLIENT_ID'),
    client_secret=os.getenv('GOOGLE_CLIENT_SECRET'),
    redirect_to='google_login'
)
app.register_blueprint(blueprint, url_prefix="/login")

# User Model
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    google_id = db.Column(db.String(256), unique=True)
    email = db.Column(db.String(256), unique=True)

@login_manager.user_loader
def load_user(user_id):
    return User.query.get(int(user_id))

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/login')
def login():
    if current_user.is_authenticated:
        return redirect(url_for('profile'))
    return render_template('login.html')

@app.route('/profile')
@login_required
def profile():
    return render_template('profile.html', user=current_user)

@app.route('/login/google')
def google_login():
    if not google.authorized:
        return redirect(url_for('google.login'))
    
    resp = google.get("/oauth2/v1/userinfo")
    if resp.ok:
        user_info = resp.json()
        user = User.query.filter_by(google_id=user_info["id"]).first()
        
        if not user:
            user = User(
                google_id=user_info["id"],
                email=user_info["email"]
            )
            db.session.add(user)
            db.session.commit()

        login_user(user)
        return redirect(url_for('profile'))

    return redirect(url_for('login'))

@app.route('/logout')
@login_required
def logout():
    logout_user()
    return redirect(url_for('home'))

if __name__ == '__main__':
    db.create_all()
    app.run(debug=True)
Step 4: Set Up Templates
index.html:

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Home</title>
</head>
<body>
    <h1>Welcome to the OAuth2 Demo</h1>
    {% if current_user.is_authenticated %}
        <p>You are logged in as {{ current_user.email }}.</p>
        <a href="{{ url_for('profile') }}">Go to Profile</a>
        <br>
        <a href="{{ url_for('logout') }}">Logout</a>
    {% else %}
        <p>You are not logged in.</p>
        <a href="{{ url_for('google.login') }}">Login with Google</a>
    {% endif %}
</body>
</html>
login.html:

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Login</title>
</head>
<body>
    <h1>Login</h1>
    <a href="{{ url_for('google.login') }}">Login with Google</a>
</body>
</html>
profile.html:

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Profile</title>
</head>
<body>
    <h1>Profile</h1>
    <p>Email: {{ user.email }}</p>
    <a href="{{ url_for('logout') }}">Logout</a>
</body>
</html>
Step 5: Add Environment Variables
Set environment variables for GOOGLE_CLIENT_ID and GOOGLE_CLIENT_SECRET:

Create a .env file or set them directly in your development environment:

export GOOGLE_CLIENT_ID="your-google-client-id"
export GOOGLE_CLIENT_SECRET="your-google-client-secret"
Install dotenv package to load environment variables in development:


pip install python-dotenv
Update the app.py to load from .env in development:


from dotenv import load_dotenv
load_dotenv()
Step 6: Deployment to Heroku
Create requirements.txt:

To install all necessary libraries on Heroku:


pip freeze > requirements.txt
Example requirements.txt:

Flask==2.1.0
Flask-Dance==5.1.0
Flask-Login==0.5.0
python-dotenv==0.19.2
SQLAlchemy==1.4.0
gunicorn==20.1.0
Create Procfile:

This is required by Heroku to run the app:

makefile

web: gunicorn app:app
Deploy to Heroku:

Create a Git repository:


git init
git add .
git commit -m "Initial commit"
Create a Heroku app:

heroku create
Deploy your code to Heroku:

git push heroku master
Set the Google API keys on Heroku:

heroku config:set GOOGLE_CLIENT_ID=your_google_client_id
heroku config:set GOOGLE_CLIENT_SECRET=your_google_client_secret
Open the app:

heroku open
Step 7: Facebook OAuth (Optional)
You can follow a similar procedure to set up Facebook OAuth2 using Flask-Dance. Just create a Facebook App via Facebook Developer Console, retrieve your App ID and App Secret, and adjust the code accordingly:


from flask_dance.contrib.facebook import make_facebook_blueprint
Then, replace the Google blueprint with the Facebook one.

By following these steps, you can integrate OAuth2 authentication into your Flask app using Google (or Facebook) and deploy it to a cloud platform like Heroku.

##  4.Develop a recommendation system using Flask that suggests content to users based on their preferences.

In [ ]:
To develop a content recommendation system using Flask, we'll create a basic web app that provides suggestions based on users' preferences. The recommendation logic can use a simple collaborative filtering or content-based filtering algorithm. For this example, we'll use a simple content-based filtering approach to suggest content based on user preferences.

Steps:
Setup Flask Application.
Create a data model for content and users' preferences.
Implement content-based recommendation.
Display recommendations on a web page.

Step 1: Set Up the Flask Application

app.py (Main Flask Application):

from flask import Flask, render_template, request
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

app = Flask(__name__)

# Load content data
content_data = pd.read_csv('content_data.csv')

# Preprocessing: Use TF-IDF to vectorize the content
tfidf = TfidfVectorizer(stop_words='english')
content_data['content'] = content_data['content'].fillna('')
tfidf_matrix = tfidf.fit_transform(content_data['content'])

# Compute cosine similarity between content items
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Index for quick lookup
indices = pd.Series(content_data.index, index=content_data['title']).drop_duplicates()

# Function to get recommendations
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]  # Top 5 similar contents
    content_indices = [i[0] for i in sim_scores]
    return content_data['title'].iloc[content_indices]

# Flask Routes
@app.route('/')
def home():
    return render_template('index.html')

@app.route('/recommend', methods=['POST'])
def recommend():
    title = request.form['title']
    recommendations = get_recommendations(title)
    return render_template('recommend.html', recommendations=recommendations, title=title)

if __name__ == '__main__':
    app.run(debug=True)


Step 2: Create HTML Templates
1.index.html:

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Content Recommendation</title>
</head>
<body>
    <h1>Welcome to the Content Recommendation System</h1>
    <form action="/recommend" method="POST">
        <label for="title">Enter Content Title:</label>
        <input type="text" id="title" name="title" required>
        <input type="submit" value="Get Recommendations">
    </form>
</body>
</html>
2.recommend.html:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Recommendations</title>
</head>
<body>
    <h1>Recommendations for {{ title }}</h1>
    <ul>
        {% for item in recommendations %}
            <li>{{ item }}</li>
        {% endfor %}
    </ul>
    <a href="/">Back to Home</a>
</body>
</html>
Step 3: Create Content Data
Prepare a CSV file (content_data.csv) with the content and its description. Here’s an example:

title,content
Content A,"This is a great article about machine learning."
Content B,"This blog post talks about deep learning and AI."
Content C,"An article explaining the basics of machine learning."
Content D,"A tutorial on how to get started with deep learning."
Content E,"Learn about artificial intelligence and its applications."


Step 4: Run the Application
python app.py
